## AI Deepfake Detection - EDA

## Imports

In [34]:
import pandas as pd
import json
import os
from IPython.display import Video

## Load the training data

In [32]:
# Helper functions
def load_metadata(metadata_path):
    """Load metadata from a JSON file and return a DataFrame."""
    with open(metadata_path) as f:
        metadata = json.load(f)
    df = pd.DataFrame.from_dict(metadata, orient='index')
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'video_filename'}, inplace=True)
    return df

def play_video(video_filename, video_folder='../data/train_sample_videos'):
    """Play a video given its filename."""
    video_path = os.path.join(video_folder, video_filename)
    if os.path.exists(video_path):
        return Video(video_path,width=320, height=320)
    else:
        print(f"Video file {video_filename} does not exist.")

In [5]:
    # Set the path for the metadata file
    metadata_path = '../data/train_sample_videos/metadata.json'

    # Load metadata into a DataFrame
    df = load_metadata(metadata_path)

In [6]:
df

,video_filename,label,split,original
0,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
1,aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
2,abarnvbtwb.mp4,REAL,train,None
3,abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
4,abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4
...,...,...,...,...
395,etejaapnxh.mp4,FAKE,train,wtreibcmgm.mp4
396,etmcruaihe.mp4,FAKE,train,afoovlsmtx.mp4
397,etohcvnzbj.mp4,FAKE,train,bdnaqemxmr.mp4
398,eudeqjhdfd.mp4,REAL,train,None


In [35]:
play_video("aapnvogymq.mp4")